In [ ]:
import json
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

# ✅ .env 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# ✅ 프롬프트 생성 함수
def make_prompt(cleaned_sentence):
    return f"""
다음 조건을 반드시 지켜서 문장을 정제해줘:

1. 입력 문장의 어순, 구조, 단어는 절대 바꾸지 마.
2. 띄어쓰기, 맞춤법, 오탈자만 고쳐.
3. 특히 종결어미가 이상하거나 특이하면 자연스럽게 고쳐.
   (예: 다 함? → 다 했어? / 감동했구만 → 감동했구나 / 안할런다 → 안 할래. / 하는거? → 하는 거야? / 꺼지셈 → 꺼져)
4. 두 문장처럼 보이면, 자연스러운 위치에 마침표를 추가해.
5. 문맥 파악이 어렵거나, 수정이 애매한 경우에는 고치지 말고 그대로 둬.
6. 표준어를 구어체로 줄인 말과 인터넷 용어의 줄임말은 다음 기준에 따라 다르게 처리해:
   - '솔까', '갠플' 등 인터넷에서만 통용되는 말들은 반드시 그대로 유지해.
   - '걍' → '그냥', '함 보자' → '한 번 보자', '구래' → '그래', '마이 아파' → '많이 아파'처럼
     기존 표준어를 축약하거나 잘못 쓴 표현은 표준어로 자연스럽게 고쳐.
   ❗ 의미가 바뀔 수 있거나 문맥이 불분명한 경우에는 고치지 마.
7. 존댓말/반말을 불필요하게 바꾸지 마.
8. 입력 문장은 반드시 온전한 문장이 아닐 수도 있어.
   채팅체 특성상 문장이 잘려 있을 수 있으니, 억지로 문장으로 만들려고 하지 마.

✅ 오직 **수정된 문장만** 출력해.  
**앞에 ‘입력:’, ‘출력:’ 같은 라벨은 절대 붙이지 마.**  
**입력이 빈 문자열인 경우, 빈 문자열만 출력하고 쌍따옴표("")는 출력하지 마.**
""" 

# ✅ GPT 호출 함수
def refine_with_chatgpt(text, model="gpt-4", max_retries=3):
    prompt = make_prompt(text)
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "너는 문장의 띄어쓰기와 맞춤법 등을 정제하는 비서야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.0
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ 오류 발생: {e} (재시도 {attempt + 1}/{max_retries})")
            time.sleep(2)
    return text

# ✅ 파일 경로
input_file = "sample_cleaned.json"
output_file = "sample_final.json"
BATCH_SIZE = 200

# ✅ 입력 파일 로드
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 기존 결과 불러오기
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        existing_data = {item["id"]: item for item in json.load(f)}
else:
    existing_data = {}

In [29]:
print("전체 데이터 개수:", len(data))
print("처음 5개 항목:")
for i in range(5):
    print(data[i])

전체 데이터 개수: 201447
처음 5개 항목:
{'time': '10:37:00', 'participantID': 'P01', 'utterance': '몇시까지먹엇는데?', 'cleaned': '몇시까지먹엇는데?', 'emotion_chunks': {}, 'extracted_emoticons': [], 'id': 1}
{'time': '10:45:00', 'participantID': 'P02', 'utterance': '저때 집 도착해ㅆ을 걸 상권이랑 둘이서 있을줄이야', 'cleaned': '저때 집 도착해ㅆ을 걸 상권이랑 둘이서 있을줄이야', 'emotion_chunks': {}, 'extracted_emoticons': [], 'id': 2}
{'time': '10:47:00', 'participantID': 'P01', 'utterance': '다른애들은??', 'cleaned': '다른애들은?', 'emotion_chunks': {'[의문]': ['??']}, 'extracted_emoticons': [], 'id': 3}
{'time': '10:48:00', 'participantID': 'P02', 'utterance': '애들 걍 갔음 오빠가고나서 그래서 혜명이랑 얘기 했는데 나랑 진짜 안맞음', 'cleaned': '애들 걍 갔음 오빠가고나서 그래서 혜명이랑 얘기 했는데 나랑 진짜 안맞음', 'emotion_chunks': {}, 'extracted_emoticons': [], 'id': 4}
{'time': '10:49:00', 'participantID': 'P02', 'utterance': '이렇게 얘기해서 개빡쳤었음 내가 제ㅔㅔㅔㅔㅔㅔ일 싫어하는 스타일 표리부동', 'cleaned': '이렇게 얘기해서 개빡쳤었음 내가 제ㅔㅔㅔㅔㅔㅔ일 싫어하는 스타일 표리부동', 'emotion_chunks': {}, 'extracted_emoticons': [], 'id': 5}


In [ ]:
# ✅ 정제 및 저장 (배치 단위)
for start in range(0, len(data), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(data))
    batch = data[start:end]
    refined_batch = []

    for item in batch:
        item_id = item.get("id")
        if not item_id:
            continue

        # 이미 정제된 항목이면 기존 데이터 유지
        if item_id in existing_data and existing_data[item_id].get("refined", "").strip():
            print(f"⏩ 건너뜀: ID {item_id} → 이미 정제됨")
            refined_batch.append(existing_data[item_id])
            continue

        cleaned = item.get("cleaned", "").strip()
        if cleaned:
            refined = refine_with_chatgpt(cleaned).strip('"')
            item["refined"] = refined
            print(f"✅ 새로 정제: ID {item_id} → {refined}")
        else:
            item["refined"] = ""

        refined_batch.append(item)

    # 기존 데이터 갱신
    for item in refined_batch:
        existing_data[item["id"]] = item

    # 저장
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(list(existing_data.values()), f, ensure_ascii=False, indent=2)

    print(f"💾 {start}~{end}번까지 저장 완료")

print(f"🎉 전체 정제 완료 및 저장됨 → {output_file}")

⏩ 건너뜀: ID 1 → 이미 정제됨
⏩ 건너뜀: ID 2 → 이미 정제됨
⏩ 건너뜀: ID 3 → 이미 정제됨
⏩ 건너뜀: ID 4 → 이미 정제됨
⏩ 건너뜀: ID 5 → 이미 정제됨
⏩ 건너뜀: ID 6 → 이미 정제됨
⏩ 건너뜀: ID 7 → 이미 정제됨
⏩ 건너뜀: ID 8 → 이미 정제됨
⏩ 건너뜀: ID 11 → 이미 정제됨
⏩ 건너뜀: ID 12 → 이미 정제됨
⏩ 건너뜀: ID 13 → 이미 정제됨
⏩ 건너뜀: ID 14 → 이미 정제됨
⏩ 건너뜀: ID 15 → 이미 정제됨
⏩ 건너뜀: ID 16 → 이미 정제됨
⏩ 건너뜀: ID 17 → 이미 정제됨
⏩ 건너뜀: ID 18 → 이미 정제됨
⏩ 건너뜀: ID 20 → 이미 정제됨
⏩ 건너뜀: ID 21 → 이미 정제됨
⏩ 건너뜀: ID 23 → 이미 정제됨
⏩ 건너뜀: ID 24 → 이미 정제됨
⏩ 건너뜀: ID 26 → 이미 정제됨
⏩ 건너뜀: ID 27 → 이미 정제됨
⏩ 건너뜀: ID 28 → 이미 정제됨
⏩ 건너뜀: ID 29 → 이미 정제됨
⏩ 건너뜀: ID 30 → 이미 정제됨
⏩ 건너뜀: ID 31 → 이미 정제됨
⏩ 건너뜀: ID 32 → 이미 정제됨
⏩ 건너뜀: ID 33 → 이미 정제됨
⏩ 건너뜀: ID 34 → 이미 정제됨
⏩ 건너뜀: ID 35 → 이미 정제됨
⏩ 건너뜀: ID 36 → 이미 정제됨
⏩ 건너뜀: ID 37 → 이미 정제됨
⏩ 건너뜀: ID 38 → 이미 정제됨
⏩ 건너뜀: ID 39 → 이미 정제됨
⏩ 건너뜀: ID 40 → 이미 정제됨
⏩ 건너뜀: ID 41 → 이미 정제됨
⏩ 건너뜀: ID 42 → 이미 정제됨
⏩ 건너뜀: ID 43 → 이미 정제됨
⏩ 건너뜀: ID 44 → 이미 정제됨
⏩ 건너뜀: ID 45 → 이미 정제됨
⏩ 건너뜀: ID 46 → 이미 정제됨
⏩ 건너뜀: ID 47 → 이미 정제됨
⏩ 건너뜀: ID 48 → 이미 정제됨
⏩ 건너뜀: ID 49 → 이미 정제됨
⏩ 건너뜀: ID 50 → 이미 정제됨
⏩ 건너뜀: ID 51 → 이미 

KeyboardInterrupt: 

In [33]:
from pprint import pprint

pprint(existing_data)

{1: {'cleaned': '몇시까지먹엇는데?',
     'emotion_chunks': {},
     'extracted_emoticons': [],
     'id': 1,
     'participantID': 'P01',
     'refined': '몇 시까지 먹었는데?',
     'time': '10:37:00',
     'utterance': '몇시까지먹엇는데?'},
 2: {'cleaned': '저때 집 도착해ㅆ을 걸 상권이랑 둘이서 있을줄이야',
     'emotion_chunks': {},
     'extracted_emoticons': [],
     'id': 2,
     'participantID': 'P02',
     'refined': '저 때 집에 도착했을 걸, 상권이랑 둘이서 있을 줄이야.',
     'time': '10:45:00',
     'utterance': '저때 집 도착해ㅆ을 걸 상권이랑 둘이서 있을줄이야'},
 3: {'cleaned': '다른애들은?',
     'emotion_chunks': {'[의문]': ['??']},
     'extracted_emoticons': [],
     'id': 3,
     'participantID': 'P01',
     'refined': '다른 애들은?',
     'time': '10:47:00',
     'utterance': '다른애들은??'},
 4: {'cleaned': '애들 걍 갔음 오빠가고나서 그래서 혜명이랑 얘기 했는데 나랑 진짜 안맞음',
     'emotion_chunks': {},
     'extracted_emoticons': [],
     'id': 4,
     'participantID': 'P02',
     'refined': '애들은 걍 갔음. 오빠가 가고 나서 그래서 혜명이랑 얘기 했는데, 나랑 진짜 안 맞음.',
     'time': '10:48:00',
     'utterance': '애들 걍 갔음 